# Computing turbulence kinetic energy from Code_Saturne time moments
In this tutorial, we calculate the turbulence kinetic energy defined as
$$k = \frac{1}{2}\left(\overline{u'u'} + \overline{v'v'} + \overline{w'w'}\right)$$
The time moments in Code_Saturne are defined in the file  `data/src/cs_user_parameters.c` in function `cs_user_time_moments`. In this case, we have computed the mean square velocity $\overline{u u}$ (variable 'UU_mean') and the mean velocity $\bar{u}$ (Variable 'U_mean'). This can be used to calculate the variances using
$$\overline{u'u'} = \overline{u u} - \bar{u}\bar{u}$$
Below we compute and plot $k$.


In [1]:
# Load data in the same manner as reading_cs
import pyvista as pv

fn = 'case/postprocessing/RESULTS_FLUID_DOMAIN.case'

reader = pv.get_reader(fn)
reader.set_active_time_value(300)

data = reader.read()['Fluid domain']

data.cell_data


pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)

In [2]:
# Get mean velocity data
Vel_mean = data.cell_data['U_mean']

# x component
U_mean = Vel_mean[:,0]
# y component
V_mean = Vel_mean[:,1]
# z component
W_mean = Vel_mean[:,2]


In [3]:
# Set symmetric tensor with $\langle u u \rangle$
Vel2_tensor = data.cell_data['UU_mean']

#Layout of symmetric tensor below
# xx component
UU_mean = Vel2_tensor[:,0]
# yy component
VV_mean = Vel2_tensor[:,1]
# zz component
WW_mean = Vel2_tensor[:,2]
# xy component
UV_mean = Vel2_tensor[:,3]
# xz component
UW_mean = Vel2_tensor[:,4]
# yz component
VW_mean = Vel2_tensor[:,5]


In [4]:
# Compute variances
uu = UU_mean - U_mean*U_mean

vv = VV_mean - V_mean*V_mean

ww = WW_mean - W_mean*W_mean

# Compute k
k= 0.5*(uu + vv + ww)

# Reassign k to cell_data
data.cell_data['k'] = k

data.cell_data


pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)
    k                       float32    (3140920,)

In [5]:
# Plot k the same as reading_cs example
p = pv.Plotter(window_size=(500,500))
plane_slice = data.slice(origin=(0,0,0),
                         normal='x')

p.add_mesh(plane_slice,
           cmap='bwr',
           scalars='k')

p.view_zy()
p.add_axes()
p.show()

Widget(value='<iframe src="http://localhost:43159/index.html?ui=P_0x7b7d9c014e90_0&reconnect=auto" class="pyvi…